In [1]:
# Import modules
import pandas as pd
import numpy as np
import re
from functools import reduce

# Set pandas number format
pd.options.display.float_format = "{:,.2f}".format

# Load and clean data


In [2]:
dir_path = "../../data/exiobase/_ixi/IOT_2014_ixi/"
rexia_dir_path = "../../data/exiobase/REX/rexia-data/converted/"

In [3]:
# Import A, Y and x from Exiobase 3.8
# ! 'A' is a large datasets, mind load time
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
# Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
x = pd.read_csv(f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1])["indout"]
# Import F, F_y (extensions) from both satellite and impacts
F_satellite = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y_satellite = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit_satellite = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)
F_impact = pd.read_csv(f"{dir_path}impacts/F.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_y_impact = pd.read_csv(f"{dir_path}impacts/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_unit_impact = pd.read_csv(f"{dir_path}impacts/unit.txt", delimiter="\t", header=0, index_col=[0])
# ! Also import Q (extension) from REXIA (Cabernard & Pfister, 2022)
Q = pd.read_csv(f"{rexia_dir_path}Q_REXIA.csv", header=[0, 1], index_col=[0])

In [4]:
# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()
FD_sector_labels = Y.columns.get_level_values(level=1).unique()

In [5]:
# Some sector labels of Q do not match those of Exiobase
wrong_labels = [
    '"Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)"',
    '"Manufacture of furniture; manufacturing n.e.c. (36)"',
    '"Manufacture of gas; distribution of gaseous fuels through mains"',
    '"Manufacture of wearing apparel; dressing and dyeing of fur (18)"',
    '"Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)"',
    '"Mining of coal and lignite; extraction of peat (10)"',
    '"Public administration and defence; compulsory social security (75)"',
    '"Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)"',
    '"Supporting and auxiliary transport activities; activities of travel agencies (63)"',
    '"Tanning and dressing of leather; manufacture of luggage, handbags, saddlery, harness and footwear (19)"',
]
correct_labels = [
    "Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)",
    "Manufacture of furniture; manufacturing n.e.c. (36)",
    "Manufacture of gas; distribution of gaseous fuels through mains",
    "Manufacture of wearing apparel; dressing and dyeing of fur (18)",
    "Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)",
    "Mining of coal and lignite; extraction of peat (10)",
    "Public administration and defence; compulsory social security (75)",
    "Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)",
    "Supporting and auxiliary transport activities; activities of travel agencies (63)",
    "Tanning and dressing of leather; manufacture of luggage, handbags, saddlery, harness and footwear (19)",
]

sectors_mapping = {wrong_labels[i]: correct_labels[i] for i, _ in enumerate(wrong_labels)}
Q.rename(columns=sectors_mapping, inplace=True)

# make sure that both indexes are the same
sorted(sector_labels) == sorted(Q.columns.get_level_values(level=1).unique())

True

In [6]:
# Aggregate countries so as to match the region clusters of Exiobase

# Use the mapping file to determine which countries is associated to which cluster
# And also change to country code instead of the full country name
countries_mapping = pd.read_csv("./countries_mapping.csv", header=None)
countries_mapping.set_index(0, inplace=True)

# check that each country is associated with a region associated in the csv file
for country in list(Q.columns.get_level_values(0).unique()):
    if len(countries_mapping.loc[country, 1]) != 2:
        print(country)

# Assign each country to its region
clusters = dict()
for code in countries_mapping[1]:
    clusters[code] = []
for country in list(Q.columns.get_level_values(0).unique()):
    clusters[countries_mapping.loc[country, 1]].append(country)

# check that we have the right number of clusters
print((len(clusters) == 49) & (sorted(region_labels) == sorted(clusters.keys())))

True


In [7]:
# Update Q with the new region and sector labels
new_Q = []
for code, countries in clusters.items():
    res = Q.loc[:, clusters[code]].groupby(level=1, axis=1).sum().loc[:, sector_labels]
    res = pd.concat([res], keys=[code], axis=1)
    new_Q.append(res)

new_Q = pd.concat(new_Q, axis=1)
new_Q.columns.set_names(["region", "sector"], inplace=True)
new_Q.index.set_names(["stressor"], inplace=True)
del Q
new_Q

region                                                                    AT   
sector                                             Cultivation of paddy rice   
stressor                                                                       
BGS mining data [t]                                                     0.00  \
Land use area of mining ? primary commodity all...                      0.00   
Land-use related biodiversity loss of mining ? ...                      0.00   
Land use area of mining ? monetary allocation [...                      0.00   
Land-use related biodiversity loss of mining ? ...                      0.00   
Total material footprint [kt]                                           0.00   
Climate change impacts [kg CO2-eq]                                      0.00   
Particulate-matter related health impacts [DALYs]                       0.00   
Blue water consumption [Mio. m3]                                        0.00   
Water stress [Mio. m3 H2O-eq]                                           0.00   
Total land use area based on EXIOBASE3 [km2]                            0.00   
Total land-use related biodiversity loss based ...                      0.00   
Total land use area based on FAOSTAT for agricu...                      0.00   
Total land-use related biodiversity loss based ...                      0.00   

region                                                                    
sector                                             Cultivation of wheat   
stressor                                                                  
BGS mining data [t]                                                0.00  \
Land use area of mining ? primary commodity all...                 0.00   
Land-use related biodiversity loss of mining ? ...                 0.00   
Land use area of mining ? monetary allocation [...                 0.00   
Land-use related biodiversity loss of mining ? ...                 0.00   
Total material footprint [kt]                                  1,632.32   
Climate change impacts [kg CO2-eq]                       564,031,974.79   
Particulate-matter related health impacts [DALYs]                237.72   
Blue water consumption [Mio. m3]                                   0.79   
Water stress [Mio. m3 H2O-eq]                                      1.00   
Total land use area based on EXIOBASE3 [km2]                   3,118.04   
Total land-use related biodiversity loss based ...         5,138,529.87   
Total land use area based on FAOSTAT for agricu...           304,645.00   
Total land-use related biodiversity loss based ...         5,250,904.40   

region                                                                                
sector                                             Cultivation of cereal grains nec   
stressor                                                                              
BGS mining data [t]                                                            0.00  \
Land use area of mining ? primary commodity all...                             0.00   
Land-use related biodiversity loss of mining ? ...                             0.00   
Land use area of mining ? monetary allocation [...                             0.00   
Land-use related biodiversity loss of mining ? ...                             0.00   
Total material footprint [kt]                                              3,133.72   
Climate change impacts [kg CO2-eq]                                   818,868,108.21   
Particulate-matter related health impacts [DALYs]                            364.99   
Blue water consumption [Mio. m3]                                               6.86   
Water stress [Mio. m3 H2O-eq]                                                  8.68   
Total land use area based on EXIOBASE3 [km2]                               4,459.47   
Total land-use related biodiversity loss based ...                     8,824,797.53   
Total land use area based on FAOSTAT for agricu...        

# Compute Leontief


In [8]:
# Create an identity matrix the same order
Id = np.identity(len(region_and_sector_labels))

# Compute leontief inverse matrix in the Demand-pull model
L_values = np.linalg.inv((Id - A))
L = pd.DataFrame(data=L_values, index=region_and_sector_labels, columns=region_and_sector_labels)

# compute extension intensity
f_satellite = F_satellite / x.T
f_satellite = f_satellite.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

f_impact = F_impact / x.T
f_impact = f_impact.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

# compute extension intensity
f_rexia = new_Q / x.T
f_rexia = f_rexia.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

# Supply chain of copper


In [9]:
# Find all the sectors that contain the word 'nickel'
MASK = sector_labels.str.contains("copper", flags=re.IGNORECASE, regex=True)
idx = sector_labels[MASK]
print(*list(zip(sector_labels.get_indexer_for(idx), idx)), sep="\n")

(25, 'Mining of copper ores and concentrates')
(79, 'Copper production')
(80, 'Re-processing of secondary copper into new copper')


In [10]:
# Derive index
# ! (matlab code lines 568-610)

COPPER = sector_labels[np.r_[25, 79]]

index_t = pd.IndexSlice[:, COPPER]
index_t = A.loc[index_t, :].index  # to have explicit labels instead of slice(None)

index_o = region_and_sector_labels.difference(index_t)

# Show result
index_t[:5]

MultiIndex([('AT', 'Mining of copper ores and concentrates'),
            ('BE', 'Mining of copper ores and concentrates'),
            ('BG', 'Mining of copper ores and concentrates'),
            ('CY', 'Mining of copper ores and concentrates'),
            ('CZ', 'Mining of copper ores and concentrates')],
           names=['region', 'sector'])

In [11]:
# Derive the Leontief inverse for the remaining economy
# ! (matlab code lines 612 - 615)

# Create an identity matrix the same order
Id = np.identity(len(index_o))

# get corresponding (numerical) index for index_o
ix_o = np.sort(region_and_sector_labels.get_indexer_for(index_o))

# Compute leontief inverse matrix in the Demand-pull model (for the remaining economy )
L_oo_dash = pd.DataFrame(
    data=np.zeros(shape=A.shape),
    index=region_and_sector_labels,
    columns=region_and_sector_labels,
)
L_oo_dash.iloc[ix_o, ix_o] = np.linalg.inv(Id - A.iloc[ix_o, ix_o])

# save intermediary calculation that will be reused multiple times
Y_region = Y.groupby(by="region", axis=1).sum()
Y_region = Y_region.loc[region_and_sector_labels, region_labels]  # reorder rows / cols
Y_global = Y_region.sum(axis=1)
AtoLoo = A.loc[index_t, index_o] @ L_oo_dash.loc[index_o, index_o]

In [12]:
# Show results
Y_region

region                                                        AT   BE   BG   
region sector                                                                
AT     Cultivation of paddy rice                            0.00 0.00 0.00  \
       Cultivation of wheat                                 0.00 0.10 2.20   
       Cultivation of cereal grains nec                   152.07 0.32 0.37   
       Cultivation of vegetables, fruit, nuts             803.64 0.33 1.37   
       Cultivation of oil seeds                             0.00 0.14 3.48   
...                                                          ...  ...  ...   
WM     Activities of membership organisation n.e.c. (91)    1.06 3.42 2.26   
       Recreational, cultural and sporting activities ...   9.15 2.20 3.83   
       Other service activities (93)                        1.54 3.59 0.14   
       Private households with employed persons (95)        0.17 1.36 0.02   
       Extra-territorial organizations and bodies           0.00 0.00 0.00   

region                                                      CY   CZ    DE   
region sector                                                               
AT     Cultivation of paddy rice                          0.00 0.00  0.00  \
       Cultivation of wheat                               0.01 0.21 14.69   
       Cultivation of cereal grains nec                   0.02 0.50 13.44   
       Cultivation of vegetables, fruit, nuts             0.09 6.68 98.67   
       Cultivation of oil seeds                           0.00 0.52 24.16   
...                                                        ...  ...   ...   
WM     Activities of membership organisation n.e.c. (91)  0.23 0.51  5.08   
       Recreational, cultural and sporting activities ... 3.69 8.63 13.57   
       Other service activities (93)                      0.01 0.39  2.65   
       Private households with employed persons (95)      0.16 0.10  1.41   
       Extra-territorial organizations and bodies         0.00 0.00  0.00   

region                                                       DK   EE    ES   
region sector                                                                
AT     Cultivation of paddy rice                           0.00 0.00  0.00  \
       Cultivation of wheat                                0.00 0.01  0.28   
       Cultivation of cereal grains nec                    0.01 0.03  0.15   
       Cultivation of vegetables, fruit, nuts              0.60 0.03  4.29   
       Cultivation of oil seeds                            0.83 0.00  0.03   
...                                                         ...  ...   ...   
WM     Activities of membership organisation n.e.c. (91)   2.78 0.37  4.85   
       Recreational, cultural and sporting activities ... 14.07 0.29 20.33   
       Other service activities (93)                       1.06 0.02  5.36   
       Private households with employed persons (95)       0.07 0.02 90.87   
       Extra-territorial organizations and bodies          0.00 0.00  0.00   

region                                                       FI  ...   TR   
region sector                                                    ...        
AT     Cultivation of paddy rice                           0.00  ... 0.00  \
       Cultivation of wheat                                0.01  ... 0.01   
       Cultivation of cereal grains nec                    0.03  ... 0.03   
       Cultivation of vegetables, fruit, nuts              1.28  ... 0.48   
       Cultivation of oil seeds                            0.16  ... 0.00   
...                                                         ...  ...  ...   
WM     Activities of membership organisation n.e.c. (91)   1.29  ... 1.09   
       Recreational, cultural and sporting activities ... 10.59  ... 0.12   
       Other service activities (93)                       1.64  ... 3.62   
       Private households with employed persons (95)       0.10  ... 1.00   
       Extra-territorial organizations and b

In [13]:
# Show results
Y_global

region  sector                                             
AT      Cultivation of paddy rice                                  0.00
        Cultivation of wheat                                     112.16
        Cultivation of cereal grains nec                         192.44
        Cultivation of vegetables, fruit, nuts                 1,033.94
        Cultivation of oil seeds                                  41.69
                                                                 ...   
WM      Activities of membership organisation n.e.c. (91)      9,206.81
        Recreational, cultural and sporting activities (92)   29,868.39
        Other service activities (93)                         18,556.71
        Private households with employed persons (95)          4,022.69
        Extra-territorial organizations and bodies                 0.00
Length: 7987, dtype: float64

## Choice of stressor


In [14]:
# Select stressors to be analysed
# ! (matlab code line ....)

# # Option 1: if we precisely know which stressor we want to analyse
# idx = [
#     "Domestic Extraction Used - Metal Ores - Nickel ores",
#     "Unused Domestic Extraction - Metal Ores - Nickel ores",
# ]

# # Option 2: find all the stressors that contain the word...
# stressor = "Water Withdrawal Blue - Manufacturing"
# MASK = d.index.str.contains(stressor, flags=re.IGNORECASE, regex=True)
# idx = d.loc[MASK].index
# print(*list(zip(d.index.get_indexer_for(idx), idx)), sep="\n")

# Option 3: use Q from REXIA (Cabernard & Pfister, 2022)
d = f_rexia  # choose f_impact / f_satellite / f_biodiv
stressor_labels = d.index  # .get_level_values(level=1).unique()
list(enumerate(d.index))

[(0, 'BGS mining data [t]'),
 (1, 'Land use area of mining ? primary commodity allocation [km2]'),
 (2,
  'Land-use related biodiversity loss of mining ? primary commodity allocation [global pdf]'),
 (3, 'Land use area of mining ? monetary allocation [km2]'),
 (4,
  'Land-use related biodiversity loss of mining ? monetary allocation [global pdf]'),
 (5, 'Total material footprint [kt]'),
 (6, 'Climate change impacts [kg CO2-eq]'),
 (7, 'Particulate-matter related health impacts [DALYs]'),
 (8, 'Blue water consumption [Mio. m3]'),
 (9, 'Water stress [Mio. m3 H2O-eq]'),
 (10, 'Total land use area based on EXIOBASE3 [km2]'),
 (11,
  'Total land-use related biodiversity loss based on EXIOBASE3 [10^12 global pdf]'),
 (12,
  'Total land use area based on FAOSTAT for agriculture and forestry [km2]'),
 (13,
  'Total land-use related biodiversity loss based on FAOSTAT for agriculture and forestry [10^12 global pdf]')]

In [135]:
d_i = d.iloc[np.r_[4]].sum()
stress = "biodiv"  # for the sankey (excel) data

# save intermediary calculation that will be reused multiple times
dLt = d_i @ L.loc[:, index_t]

# # ! make sure it is a column vector (= pd.Series). use sum() if necessary
d_i

region  sector                                             
AT      Cultivation of paddy rice                             0.00
        Cultivation of wheat                                  0.00
        Cultivation of cereal grains nec                      0.00
        Cultivation of vegetables, fruit, nuts                0.00
        Cultivation of oil seeds                              0.00
                                                              ... 
WM      Activities of membership organisation n.e.c. (91)     0.00
        Recreational, cultural and sporting activities (92)   0.00
        Other service activities (93)                         0.00
        Private households with employed persons (95)         0.00
        Extra-territorial organizations and bodies            0.00
Length: 7987, dtype: float64

## Link Preg Treg

$$E_{P,T} = diag\biggl(d_{i}\biggr) \times L_{all, T} \times diag\biggl(Y_{all, T} + A_{T,O} \times L'_{O,O} \times Y_{all, O}\biggr)$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [136]:
# ! (matlab code lines 618-625)

E_P_T = pd.DataFrame(
    data=np.zeros(shape=A.shape),
    index=region_and_sector_labels,
    columns=region_and_sector_labels,
)

# Compute Lall-t . diag( Yall-t +  Ato.Loo.Yall-o)
Lt_x_Yt_plus_AtoLooY_GLO = (
    L.loc[:, index_t] * (Y_global.loc[index_t] + AtoLoo @ Y_global.loc[index_o]).T
)

# Compute diag(di) . Lall-t . diag( Yall-t +  Ato.Loo.Yall-o)
E_P_T.loc[:, index_t] = (d_i * Lt_x_Yt_plus_AtoLooY_GLO.T).T.values

# sum the sectors together and keep regions disaggregated along both axis
E_Preg_Treg = E_P_T.groupby("region", axis=0).sum().groupby("region", axis=1).sum()

# reorder rows and cols
E_Preg_Treg = E_Preg_Treg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory
del Lt_x_Yt_plus_AtoLooY_GLO

# show results
E_Preg_Treg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,10.51,1.09,1.01,0.01,0.27,7.39,0.01,0.03,0.93,0.19,...,0.74,0.67,0.12,0.22,0.07,3.68,3.30,0.43,1.92,70.98
BE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BG,83.79,"3,048.74","63,269.21",3.42,78.74,"3,323.56",0.09,16.21,145.70,"6,531.18",...,"2,405.30",212.61,146.31,14.20,29.64,415.63,12.52,234.53,251.25,514.00
CY,0.46,0.82,1.26,"30,170.69",0.18,2.08,0.03,0.03,0.57,0.45,...,2.73,0.36,0.07,0.99,0.10,1.22,4.53,0.61,0.78,1.26
CZ,15.67,8.06,3.55,0.02,508.39,112.16,0.12,0.12,3.18,1.30,...,1.89,2.28,1.25,0.30,0.08,4.07,11.74,7.85,2.71,6.77
DE,10.33,19.47,5.39,0.45,29.86,439.64,0.20,0.17,21.72,4.33,...,5.39,4.05,3.98,0.67,0.91,9.76,26.48,4.12,9.99,49.89
DK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
EE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ES,122.06,"4,375.58","61,477.78",6.52,117.11,"4,060.85",0.10,5.58,"66,440.16",63.62,...,"4,648.40",377.92,58.79,26.47,0.83,610.10,38.10,433.22,432.33,881.90


## Link Treg FDreg

$$E_{T, FDreg} = diag\biggl(d_{i} \times L_{all, T}\biggr) \times \biggl(Y_{T, reg} + A_{T,O} \times L'_{O,O} \times Y_{O, reg}\biggr)$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [137]:
# ! (matlab code lines 627-629)

E_T_FDreg = pd.DataFrame(
    data=np.zeros(shape=(len(region_and_sector_labels), len(region_labels))),
    index=region_and_sector_labels,
    columns=region_labels,
)

# Compute Yrt + Ato.Loo.Yro
Yt_plus_AtoLooY_REG = Y_region.loc[index_t, :] + AtoLoo @ Y_region.loc[index_o, :]

# Compute diag(di) .  Lall-t  .  ( Yrt + Ato.Loo.Yro )
E_T_FDreg.loc[index_t, :] = (dLt * Yt_plus_AtoLooY_REG.T).T.values

# group sectors together (along the rows) and keep regions disaggregated
E_Treg_FDreg = E_T_FDreg.groupby("region", axis=0).sum().loc[region_labels]

# reorder rows and cols
E_Treg_FDreg = E_Treg_FDreg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory space
del E_T_FDreg

# Show results
E_Treg_FDreg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,"2,032.18",40.81,13.81,1.48,56.04,810.20,23.81,7.97,69.42,19.05,...,61.28,17.38,29.12,13.95,9.45,220.19,81.72,44.19,82.98,172.77
BE,365.70,"12,971.64",49.17,8.89,194.43,"4,164.59",133.31,21.21,816.07,155.91,...,501.53,84.22,211.52,160.14,210.35,"2,230.02","1,056.42",184.15,"1,481.53","1,780.39"
BG,"1,689.17","8,597.78","37,060.15",85.33,"1,058.09","15,035.13",612.81,123.76,"2,482.31",844.39,...,"28,361.76","7,167.10",897.77,"1,010.73","8,419.18","15,581.02","3,761.29","1,859.81","5,663.53","23,880.40"
CY,258.61,245.31,82.44,"15,769.13",207.78,446.77,185.74,111.96,318.79,232.67,...,447.73,323.25,335.44,236.58,194.99,475.17,444.16,277.34,427.87,"1,159.86"
CZ,87.00,49.41,10.60,1.03,923.25,854.35,18.71,4.35,63.67,17.59,...,62.53,22.86,31.41,15.16,6.78,232.51,65.56,47.61,64.63,149.15
DE,"6,735.05","4,338.21",410.50,54.95,"3,393.18","67,286.33","2,537.65",232.24,"5,561.08","1,452.07",...,"3,821.69","1,490.88","2,182.81","1,145.69",846.45,"16,611.01","6,301.80","1,554.04","5,788.58","18,823.01"
DK,0.37,0.42,0.05,0.01,0.35,6.93,12.86,0.07,0.55,0.43,...,0.51,0.25,1.95,0.16,0.11,2.64,1.10,0.28,0.75,1.58
EE,1.88,3.55,0.33,0.10,1.41,29.45,2.62,31.87,2.29,19.45,...,3.84,3.35,10.14,1.47,0.57,25.83,5.70,2.23,5.08,10.45
ES,926.93,"2,511.13",275.20,74.99,664.39,"10,771.48",414.22,76.54,"54,430.62",446.97,...,"13,414.11","14,229.12",727.57,846.13,441.25,"12,475.03","7,332.86",782.65,"16,772.04","33,582.72"


## Link Preg FDreg

$$E_{P, FDreg} = diag(d_{i}) \times L_{all, T} \times (Y_{T, reg} + A_{T,O} \times L'_{O,O} \times Y_{O, reg})$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [138]:
# ! (matlab code lines 635-640)

# Compute  Lgt . ( Yrt + Ato.Loo.Yro )
Lt_dot_Yt_plus_AtoLooY_REG = L.loc[:, index_t] @ Yt_plus_AtoLooY_REG

# Compute  diag(di) .  Lall-t . ( Yrt + Ato.Loo.Yro )
E_P_FDreg = (d_i * Lt_dot_Yt_plus_AtoLooY_REG.T).T

# group sectors together (along the rows) and keep regions disaggregated
E_Preg_FDreg = E_P_FDreg.groupby("region", axis=0).sum()

# reorder rows and cols
E_Preg_FDreg = E_Preg_FDreg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory space
del Yt_plus_AtoLooY_REG, Lt_dot_Yt_plus_AtoLooY_REG, E_P_FDreg

# show results
E_Preg_FDreg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,3.86,1.13,0.24,0.06,0.43,5.50,0.24,0.05,1.35,0.26,...,2.54,1.41,0.38,1.09,0.42,9.62,2.18,0.70,4.83,34.52
BE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BG,488.61,"2,182.52","6,997.47",19.81,299.34,"4,530.47",210.92,56.57,757.30,"1,694.95",...,"4,565.77","5,999.75",363.63,427.32,"8,076.33","6,062.90","1,393.49",540.17,"1,770.28","8,063.15"
CY,258.26,245.09,82.40,"15,742.95",206.71,445.22,185.53,111.89,318.24,232.52,...,448.35,322.94,335.06,236.69,194.82,471.13,443.50,277.31,426.74,"1,148.90"
CZ,22.06,11.03,2.23,0.24,92.08,134.97,4.10,0.84,12.74,3.66,...,13.00,4.87,6.37,3.54,1.56,46.73,14.04,12.54,15.32,35.85
DE,16.14,14.97,1.86,0.32,12.11,124.33,5.16,0.63,17.54,4.32,...,15.15,6.13,6.55,4.76,2.58,57.25,19.24,7.13,25.38,69.83
DK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
EE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ES,763.38,"3,466.75","8,164.26",36.04,483.50,"6,894.92",301.43,54.10,"17,583.72",283.90,...,"9,186.09","13,834.68",453.05,788.27,266.84,"11,559.36","2,909.47",943.53,"3,604.44","14,086.32"


## Link Tsec Psec


In [139]:
# ! (matlab code lines 642-651)

# group E_P_T regions together (along rows + columns) and keep sectors disaggregated
E_Psec_Tsec = E_P_T.groupby("sector", axis=0).sum().groupby("sector", axis=1).sum()

# reorder cols/rows and transpose
E_Psec_Tsec = E_Psec_Tsec.loc[sector_labels, sector_labels]
E_Tsec_Psec = E_Psec_Tsec.T

# delete variables that are no longer used to save memory space
del E_Psec_Tsec

# show result
E_Tsec_Psec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Link Psec Preg


In [140]:
# ! (matlab code lines 653-655)

# sum the columns of E_P_T and reshape the vector into 163 x 49
E_Psec_Preg = E_P_T.sum(axis=1).unstack(level=0)

# reorder cols/rows
E_Psec_Preg = E_Psec_Preg.loc[sector_labels, region_labels]

# show result
E_Psec_Preg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## link FDreg FSsec

$$
\begin{align}
    E_{FS, FDreg} &= diag\biggl(d_{i} \times L_{all, T}\biggr) \times Y_{T, reg}\\
    &+ diag\biggl(d_{i} \times L_{all, T} \times A_{T,O} \times L'_{O,O}\biggr) \times Y_{O, reg}\\
\end{align}
$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [141]:
# ! (matlab code lines 658-670)

# Compute diag(di . Lall-t) . Yrt
E_FS_FDreg_t = (dLt * Y_region.loc[index_t, :].T).T

# reorder cols/rows
E_FS_FDreg_t = E_FS_FDreg_t.loc[index_t, region_labels]

# Compute diag(di . Lt . Ato . Loo)  .  Yro
E_FS_FDreg_o = ((dLt @ AtoLoo) * Y_region.loc[index_o, :].T).T

# reorder cols/rows
E_FS_FDreg_o = E_FS_FDreg_o.loc[index_o, region_labels]

# join both matrices to get the full E_FS_FDreg
E_FS_FDreg = pd.concat([E_FS_FDreg_t, E_FS_FDreg_o], axis=0)

# reorder cols/rows
E_FS_FDreg = E_FS_FDreg.loc[region_and_sector_labels, region_labels]

# group regions together (along rows) and keep sectors disaggregated
E_FSsec_FDreg = E_FS_FDreg.groupby("sector", axis=0).sum()

# reorder cols/rows and transpose
E_FSsec_FDreg = E_FSsec_FDreg.loc[sector_labels, region_labels]
E_FDreg_FSsec = E_FSsec_FDreg.T

# save space by removing variables that are not used anymore
# del E_FS_FDreg_t, E_FS_FDreg_o, E_FSsec_FDreg, E_FS_FDreg

# show result
E_FDreg_FSsec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
region,,,,,,,,,,,,,,,,,,,,,
AT,0.12,9.57,22.84,120.65,15.83,0.29,1.77,3.43,19.20,21.33,...,0.86,0.26,0.99,0.99,3.54,210.30,242.99,83.03,0.29,0.00
BE,4.25,31.56,6.71,156.96,8.40,1.20,6.49,144.66,19.32,15.84,...,1.82,0.75,1.94,0.67,0.73,49.69,453.58,323.59,1.06,0.00
BG,0.49,11.67,9.99,41.19,36.76,0.15,0.91,112.87,1.69,1.50,...,3.91,0.56,3.57,0.61,0.85,10.29,454.71,32.50,0.05,0.00
CY,0.08,1.66,0.89,7.03,0.34,0.09,0.18,5.40,0.40,0.89,...,0.20,0.30,0.39,2.08,1.08,40.46,140.25,47.61,0.10,0.00
CZ,0.44,43.96,14.72,72.13,15.82,0.45,2.44,27.84,2.52,11.67,...,19.47,1.92,13.94,3.81,4.93,66.93,288.05,44.72,0.12,0.00
DE,16.08,180.94,85.30,965.56,83.46,6.98,13.64,504.08,53.90,83.72,...,21.18,8.41,59.77,3.81,7.52,"1,015.29","1,788.44",748.57,2.14,0.00
DK,0.30,84.05,33.23,45.46,9.69,0.19,0.56,16.90,7.09,10.77,...,0.46,0.38,7.12,0.28,0.43,95.52,344.63,49.22,0.23,0.00
EE,0.31,0.87,2.79,8.30,0.29,0.19,0.67,19.51,2.20,2.88,...,0.23,0.11,0.08,0.06,0.07,3.35,18.10,9.79,0.15,0.00
ES,9.93,174.28,13.44,659.69,7.97,2.13,3.17,168.50,48.72,88.75,...,35.06,8.22,56.30,6.54,9.46,492.63,"2,053.51",747.47,344.89,0.00


## Link Tsec Treg


In [142]:
# ! (matlab code lines 672-674)

# sum the rows of E_P_T and reshape the vector into 163 x 49
E_Tsec_Treg = E_P_T.sum(axis=0).unstack(level=0)

# reorder cols/rows
E_Tsec_Treg = E_Tsec_Treg.loc[sector_labels, region_labels]

# show result
E_Tsec_Treg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## link Preg vs FSsec

This section is extra compared to the Nickel analysis

$$
\begin{align}
    E_{P, FS} &= diag\biggl(d_{i}\biggr)  \times L_{all, T}\times diag\biggl(Y_{all, T} \biggr)\\
    &+ diag\biggl(d_{i}\biggr) \times L_{all, T} \times A_{T,O} \times L'_{O,O} \times  diag\biggl(Y_{all, O} \biggr)\\
\end{align}
$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [143]:
# ! (matlab code lines 857-869)

# Compute L . diag(Yt)
diagLt_Yt = L.loc[:, index_t] * Y_global.loc[index_t].T

# Compute diag(di) . L . diag(Yt)
E_P_FS_t = (d_i * diagLt_Yt.T).T

# Compute  (L Ato Loo) . diag(Yo)
LtAtoLoo_diagYo = (L.loc[:, index_t] @ AtoLoo) * Y_global.loc[index_o].T

# Compute diag(di) . (L Ato Loo) . diag(Yo)
E_P_FS_o = (d_i * LtAtoLoo_diagYo.T).T

# join both matrices to get full E_FS_FDreg
E_P_FS = pd.concat([E_P_FS_t, E_P_FS_o], axis=1)

# reorder cols/rows
E_P_FS = E_P_FS.loc[region_and_sector_labels, region_labels]

# group E_P_FS regions together (along rows + columns) and keep sectors disaggregated
E_Psec_FSsec = E_P_FS.groupby("sector", axis=0).sum().groupby("sector", axis=1).sum()

# reorder cols/rows
E_Psec_FSsec = E_Psec_FSsec.loc[sector_labels, sector_labels]

# save space by removing variables that are not used anymore
# del diagLt_Yt, LtAtoLoo_diagYo, E_P_FS_t, E_P_FS_o, E_P_FS

# show result
E_Psec_FSsec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Sankeys preparation


## Preg


In [144]:
# ! (matlab code lines 932-944)

agg_regions = {
    "Australia": region_labels == "AU",
    "Belgium": region_labels == "BE",
    "Brazil": region_labels == "BR",
    "Bulgaria": region_labels == "BG",
    "Canada": region_labels == "CA",
    "China": region_labels == "CN",
    "Germany": region_labels == "DE",
    "India": region_labels == "IN",
    "Indonesia": region_labels == "ID",
    "Japan": region_labels == "JP",
    "Mexico": region_labels == "MX",
    "Netherlands": region_labels == "NL",
    "Poland": region_labels == "PL",
    "South Korea": region_labels == "KR",
    "Russia": region_labels == "RU",
    "Spain": region_labels == "ES",
    "USA": region_labels == "US",
    # "EU27": region_labels.isin(region_labels[:28]),
    "RoW Asia Pacific": (region_labels == "WA") | (region_labels == "TW"),
    "RoW America": region_labels == "WL",
    "RoW Africa": (region_labels == "WF") | (region_labels == "ZA"),
    "RoW Middle East": region_labels == "WM",
}
agg_regions["RoW Europe"] = ~reduce(np.logical_or, list(agg_regions.values()))
# agg_regions["other"] = ~reduce(np.logical_or, list(agg_regions.values()))

Agg_Preg = pd.DataFrame(index=region_labels, columns=agg_regions.keys())
for regions in Agg_Preg.columns:
    Agg_Preg.loc[:, regions] = agg_regions[regions].astype(int)

if sum(sum(agg_regions.values())) != len(region_labels):
    print("Warning: some regions are missing in the aggregation matrix")

Agg_Preg

,Australia,Belgium,Brazil,Bulgaria,Canada,China,Germany,India,Indonesia,Japan,...,Poland,South Korea,Russia,Spain,USA,RoW Asia Pacific,RoW America,RoW Africa,RoW Middle East,RoW Europe
region,,,,,,,,,,,,,,,,,,,,,
AT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
BE,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BG,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
CZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
DE,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
EE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ES,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## FSsec


In [145]:
# ! (matlab code lines 947-963)

agg_sectors = {
    "agrifood": sector_labels[np.r_[0:19, 34:46]],
    "mining": sector_labels[19:34],
    "manufacturing": sector_labels[np.r_[46:85, 92]],
    "machinery_electronics": sector_labels[85:90],
    "auto": sector_labels[90:92],
    "electricity": sector_labels[95:109],
    "gas_water": sector_labels[109:112],
    "construction": sector_labels[112:114],
    "service": sector_labels[np.r_[114:119, 125:138]],
    "transportation": sector_labels[119:125],
    "waste": sector_labels[np.r_[93:95, 138:158]],
    "other_sec": sector_labels[158:],
}

Agg_FSsec = pd.DataFrame(index=sector_labels, columns=agg_sectors.keys())
for sectors in Agg_FSsec.columns:
    Agg_FSsec.loc[:, sectors] = sector_labels.isin(agg_sectors[sectors]).astype(int)

if sum(len(sec) for sec in agg_sectors.values()) != len(sector_labels):
    print("Warning: some sectors are missing in the aggregation matrix")

Agg_FSsec

,agrifood,mining,manufacturing,machinery_electronics,auto,electricity,gas_water,construction,service,transportation,waste,other_sec
sector,,,,,,,,,,,,
Cultivation of paddy rice,1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of wheat,1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of cereal grains nec,1,0,0,0,0,0,0,0,0,0,0,0
"Cultivation of vegetables, fruit, nuts",1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of oil seeds,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0,0,0,0,0,0,1
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0,0,0,0,0,0,1
Other service activities (93),0,0,0,0,0,0,0,0,0,0,0,1


## Psec


In [146]:
# ! (matlab code lines 886-906)

agg_production = {
    "coal": sector_labels[[19]],
    "nickel": sector_labels[[26]],
    "iron": sector_labels[np.r_[24, 71, 72]],
    "copper": sector_labels[np.r_[25, 79, 80]],
    "aluminium": sector_labels[np.r_[27, 75, 76]],
    "precious_metals": sector_labels[np.r_[28, 73, 74]],
}
all_metals = sector_labels[np.r_[23:31, 70:83]]
primary_metals = (
    set(agg_production["nickel"])
    | set(agg_production["iron"])
    | set(agg_production["copper"])
    | set(agg_production["aluminium"])
    | set(agg_production["precious_metals"])
)
agg_production["other_metals"] = all_metals.difference(primary_metals)
agg_production["non_mining"] = sector_labels.difference(
    set(all_metals) | set(agg_production["coal"])
)

Agg_Psec = pd.DataFrame(index=sector_labels, columns=agg_production.keys())
for sectors in Agg_Psec.columns:
    Agg_Psec.loc[:, sectors] = sector_labels.isin(agg_production[sectors]).astype(int)

# Show
Agg_Psec

,coal,nickel,iron,copper,aluminium,precious_metals,other_metals,non_mining
sector,,,,,,,,
Cultivation of paddy rice,0,0,0,0,0,0,0,1
Cultivation of wheat,0,0,0,0,0,0,0,1
Cultivation of cereal grains nec,0,0,0,0,0,0,0,1
"Cultivation of vegetables, fruit, nuts",0,0,0,0,0,0,0,1
Cultivation of oil seeds,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0,0,1
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0,0,1
Other service activities (93),0,0,0,0,0,0,0,1


## Tsec


In [147]:
# ! (matlab code lines ...)

# Remove the "coal" and "non_mining" columns to get Agg_Tsec
Agg_Tsec = Agg_Psec[["nickel", "iron", "copper", "aluminium", "precious_metals", "other_metals"]]
Agg_Tsec

,nickel,iron,copper,aluminium,precious_metals,other_metals
sector,,,,,,
Cultivation of paddy rice,0,0,0,0,0,0
Cultivation of wheat,0,0,0,0,0,0
Cultivation of cereal grains nec,0,0,0,0,0,0
"Cultivation of vegetables, fruit, nuts",0,0,0,0,0,0
Cultivation of oil seeds,0,0,0,0,0,0
...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0
Other service activities (93),0,0,0,0,0,0


## Sankey #1


In [148]:
# ! (matlab code lines 965-982)

# shares_Sankey_Tsec_Psec = (Agg_Psec.T @ E_Tsec_Psec.T @ Agg_Tsec) / E_Preg_Treg.sum().sum()
# shares_Sankey_Psec_Preg = (Agg_Psec.T @ E_Psec_Preg @ Agg_Preg) / E_Preg_Treg.sum().sum()
shares_Sankey_Preg_Treg = Agg_Preg.T @ E_Preg_Treg @ Agg_Preg  # / E_Preg_Treg.sum().sum()
shares_Sankey_Treg_FDreg = Agg_Preg.T @ E_Treg_FDreg @ Agg_Preg  # / E_Treg_FDreg.sum().sum()
shares_Sankey_FDreg_FSsec = Agg_Preg.T @ E_FDreg_FSsec @ Agg_FSsec  # / E_FDreg_FSsec.sum().sum()

perspectives = [
    shares_Sankey_Preg_Treg.sum(axis=1),
    shares_Sankey_Treg_FDreg.sum(axis=1),
    shares_Sankey_Treg_FDreg.sum().T,
    shares_Sankey_FDreg_FSsec.sum().T,
]
linkages = [
    shares_Sankey_Preg_Treg.stack(),
    shares_Sankey_Treg_FDreg.stack(),
    shares_Sankey_FDreg_FSsec.stack(),
]

# Show results
threshold = 0.001
print(stress, "\n")
for i in range(3):
    print(f"linkage {i+1}")
    mask = linkages[i] > threshold
    print(linkages[i][mask], "\n\n")
for i in range(4):
    print(f"perspectives {i+1}")
    mask = perspectives[i] > threshold
    print(perspectives[i][mask], "\n\n")

biodiv 

linkage 1
Australia   Australia          744,150.71
            Belgium              2,040.26
            Brazil               1,041.50
            Bulgaria             7,396.87
            Canada               3,632.28
                                  ...    
RoW Europe  RoW Asia Pacific     2,022.42
            RoW America            512.82
            RoW Africa             813.43
            RoW Middle East      3,552.54
            RoW Europe         333,818.33
Length: 440, dtype: object 


linkage 2
Australia   Australia          292,813.87
            Belgium              2,544.56
            Brazil               3,651.57
            Bulgaria               141.05
            Canada               5,529.93
                                  ...    
RoW Europe  RoW Asia Pacific    47,834.25
            RoW America          9,410.98
            RoW Africa          14,940.78
            RoW Middle East     31,949.14
            RoW Europe         423,974.83
Length: 484, dtyp

In [149]:
# Save results
metal = "copper"
year = 2014
filename = f"{metal}_{stress}_{year}_sankey_1"
mapping = {"level_0": "source_country", "level_1": "target_country", 0: "value"}
stages = ["mining", "processing", "consumption", "End-use"]

for i, _ in enumerate(linkages):
    linkages[i] = linkages[i].reset_index().rename(columns=mapping)
    linkages[i]["source_stage"] = stages[i]
    linkages[i]["target_stage"] = stages[i + 1]
    linkages[i] = linkages[i][
        ["source_stage", "source_country", "target_stage", "target_country", "value"]
    ]

pd.concat(linkages, axis=0).to_csv(f"./results/{metal}/{filename}.csv", index=False)

### Biodiversity

![Biodiv](./img/copper/copper_biodiv.png)

### Land use

![Land use](./img/copper/copper_landuse.png)

### Greenhouse gases

![ghg](./img/copper/copper_ghg.png)

### Blue water consumption

![water](./img/copper/copper_water.png)
